This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [6]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

QHSR7xVeKLQDyyW6FCHw


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [27]:
# First, import the relevant modules
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [10]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
baseURL = "https://data.nasdaq.com/api/v3/datasets/FSE/"
dataset ='AFX_X'
r = requests.get(url = baseURL+ dataset + '/data.json', params = {"api_key": API_KEY} )
print(r.url)

https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=QHSR7xVeKLQDyyW6FCHw


In [58]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data=json.loads(r.text)
#print (data['dataset_data'])

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [40]:
# data_2017 = [d for d in data["dataset_data"]["data"] if "2017" in d[0]]
# ans #1
timerange={ 
        "start_date":"2017-01-01", 
        "end_date":"2017-12-31",
        "api_key":API_KEY
      }
r_t = requests.get(url = baseURL+ dataset + '/data.json', params = timerange )


In [41]:
# ans #2
data2017 = json.loads(r_t.text)

In [50]:
# ans #3
columns = data2017['dataset_data']['column_names']
i_open = columns.index('Open')
open_prices = [d[i_open] for d in data2017['dataset_data']["data"] if d[i_open]]
highest_open = max (open_prices)
lowest_open = min (open_prices)
print ("highest opening:", highest_open)
print ("lowest opening:", lowest_open)

highest opening: 53.11
lowest opening: 34.0


In [51]:
# ans #4
i_high = columns.index('High')
i_low = columns.index('Low')
change_prices = [d[i_high]-d[i_low] for d in data2017['dataset_data']["data"] if d[i_high] and d[i_low]]
largest_change = max (change_prices)
print ("largest change on price: ", largest_change)

largest change on price:  2.8100000000000023


In [53]:
# ans #5
i_close = columns.index('Close')
close_prices = [d[i_close] for d in data2017['dataset_data']["data"] if d[i_close]]
largest_change_any_2_day = max (close_prices) - min (close_prices)
largest_change_2_seq_day = max ([abs(close_prices[i]-close_prices[i+1]) for i in range(len(close_prices)-1)])
print ("largest change in any 2 day: ", largest_change_any_2_day)
print ("largest change in any 2 days in a row", largest_change_2_seq_day)

largest change in any 2 day:  19.03
largest change in any 2 days in a row 2.559999999999995


In [56]:
# ans #6
i_volume = columns.index('Traded Volume')
volume = [d[i_volume] for d in data2017['dataset_data']["data"] if d[i_volume]]
avg_volume = sum(volume)/len(volume)
print ("the average trading volumn: ", avg_volume)

the average trading volumn:  89124.33725490196


In [57]:
# ans #7
sort_volume = sorted(volume)
i_median = round(len(sort_volume)/2)
median_volume = sort_volume[i_median]
print("median volume is: ",median_volume)

median volume is:  76600.0
